#### Please input your username (the one used for login) in the next cell. Then, run it with Shift+Enter (Shift+Return).

In [ ]:
username = 'username'

# Cross-Account vs Intra-Account Rules, and What is Root?

In particular, what does the following mean when used as the principal in an IAM resource policy?

"Principal": {"AWS": ["arn:aws:iam::111122223333:root]}

## Introduction

This lab examines the difference between IAM vs AWS Resource based policies. In particular, we seek to understand the policy evaluation logic for S3 buckets with cross account access. For a refresher on IAM basics, see [Reference Policies Evaluation Logic](https://docs.aws.amazon.com/IAM/latest/UserGuide/reference_policies_evaluation-logic.html), which is valid for when the IAM Principal and S3 Resource are in the same AWS account.

To summarize the above, if an action is allowed by an identity-based policy, a resource-based policy, or both, then AWS allows the action. An explicit deny in either of these policies overrides the allow.

The situation changes for [cross account access](https://aws.amazon.com/premiumsupport/knowledge-center/cross-account-access-s3/). In this case, access must be explicitly allowed in both the principal's AWS access policy and the resource policy. Unfortunately, the latter reference does not mention the confused deputy issue for cross-account access which occurs when the trusted account is a 3rd party SaaS vendor. As a result, many vendors which operate on customer's S3 buckets do so insecurely.

For this lab, we will assume both AWS accounts are owned by the same entity and will leave confused deputy issues for Lab 4 - Direct Access vs Assume Role: Granting cross account access to resources.

<img src="s3-cross-account.png" class="left"/>

Granting permissions for Principal-A to access Resource-B when both are in the same account can be done by giving Principal-A a permissions policy to access Resource-B. Alternatively, cross-account access could be granted in a resource policy such as the following bucket policy.

In this lab, there are S3 buckets we will refer to as bucket 1 through 5. Buckets 1, 2 and 4 are in the same accounts as the roles, and 3 and 5 are in a different account.
All roles shown below will have similar permissions policies which allows them to access generic S3 resources, and we'll be showing those along the way.

This lab would require having admin access to two AWS accounts, but in this case we have already created all the necessary resources.

These include an IAM user that can assume the necessary roles to perform all the actions, and the buckets we'll be accessing. But how are we going to do that? Well, we've written a function for this. It's in a script called aws_run_as.sh, and we'll run it as necessary. But first, we need to set up our credentials. To do this, simply run the next cell and everything will be set up for you. Remember, Shift+Enter / Shift+Return to run cells.

In [ ]:
!aws configure set aws_access_key_id  ${username}_AWS_KEY --profile lab2
!aws configure set aws_secret_access_key ${username}_AWS_SECRET_KEY --profile lab2
!aws configure set region us-east-1
!aws configure set output json
!chmod a+x aws_run_as.sh
roleA = username+'-role-A'
roleB = username+'-role-B'

Our function can be called like this:

```!./aws_run_as.sh --profile [profile] [role] (aws command args)```

This does the magic of using user/service credentials in "profile" to 
assume "role" and then run commands. Without this function, you have to get the response 
for assume-role and put them into environment variables or ~/.aws/credentials profile each time
as described [here](https://docs.aws.amazon.com/cli/latest/userguide/cli-configure-role.html). 
This is a very new beta script so drop into a bash shell if anything goes wrong.

Now, let's test it!

In [ ]:
!./aws_run_as.sh --profile lab2 $roleA sts get-caller-identity

This should return an assumed role with the name username-role-A, the account it's in, and its ARN. If it doesn't, check your username was spelled correctly.

# Exercises

## 1. Before we begin
So far, policies and roles were created. 
When we create a role, we must include the assume-role 
trust policy which says who can assume the role. Then, we either attach (if it's an AWS policy) or put (if it's our policy) a permissions policy, which says what that role can do, and where.

## 2. Check that a role with no permissions can access a bucket with the right bucket policy

Let's review bucket1's policy.

This policy is allowing the root of the account and roleB access to bucket1.
But what does that mean in practice? Take into account that neither of the roles has explicit permission in their policies to access bucket1.

In [ ]:
!./aws_run_as.sh --profile lab2 $roleA s3 ls s3://{bucket1.s}

Expect

An error occurred (AccessDenied) when calling the ListObjects operation: Access Denied

In [ ]:
!./aws_run_as.sh --profile lab2 $roleB s3 ls s3://$username-bucket-1

### Conclusion
Allowing "root" on an S3 policy does not grant access to all principals in the account. However, explicitly allowing a role in the S3 policy permits access even if the role has no attached permissions. This is what we mean when we say "If the principal and the resource are in the same account, permission is the union of policies attached to the resource and principal."

## 3. Can a role with permissions access a bucket with no bucket policy?

Now, let's review roleA's permission policy regarding bucket2.

Note that bucket2 has no bucket policy attached to it, meaning that it is neither allowing nor blocking roleA's access to it. Do you think we'll be able to access it?
Let's try.

In [ ]:
!./aws_run_as.sh --profile lab2 $roleA s3 ls s3://$username-bucket-

### Conclusion

We confirmed that an IAM policy attached to a role is all that is required to access a bucket, further supporting the "Union within an account" rule.

## 4. Can a role with explicit IAM policy permission to access mybucket3 in a different account access it?

Now, we want to test if we can access bucket3 in a different account by putting a specific role policy that allows us to do just that.

Let's review mybucket3's policy.

This policy is very similar to bucket1's policy, but it's located in a different account, so it behaves differently.

In this case, roleA has permission to access, and roleB doesn't. It shouldn't matter though, since roleB is explicitly allowed in the bucket policy. Or should it?

Let's test it out with both roles.

In [ ]:
!./aws_run_as.sh --profile lab2 $roleA s3 ls s3://$username-bucket-3

In [ ]:
!./aws_run_as.sh --profile lab2 $roleB s3 ls s3://$username-bucket-

What happened? We can access from roleA, which isn't allowed in the bucket policy, but not from roleB, which is.

This is because bucket3 trusts the root of our account, which means that it trusts the admin of our account to assign s3 permissions, INCLUDING accessing other accounts.

This result is not consistent with the result when the role and s3 bucket are in the same account. For cross-account, even when an s3 bucket explicitly names a resource from another account, the role must also have the permission attached.

### Conclusion

We couldn't access a bucket in another account without an explicit allow inside the bucket policy AND the IAM permissions policy. That is perfectly fine, since if it let us, we could access any bucket in any account just by knowing the bucket's name.

# 5. The necessity of explicit Deny statements

What if you wish to only allow roleA and no other principal access to mybucket4? You might try to apply granular roles to all principals in your account so that you never granted access to resource * for s3 operations to any principal. This is difficult to enforce. A better way is to apply an explicit Deny to all principals except roleA in bucket4's policy.

For that, we'll need to review bucket4's policy

In this case, both roles have permissions in their policies to access bucket4, but we should only be able to do it from roleA. Let's test.

In [ ]:
!./aws_run_as.sh --profile lab2 $roleB s3 ls s3://$username-bucket-

In [ ]:
!./aws_run_as.sh --profile lab2 $roleA s3 ls s3://$username-bucket-

### Conclusion
If we need to, we can create a bucket policy that denies permission to all principals, except the one we want, which is good for scalability and security best practices

# Congratulations! 

You've completed Lab2. Or have you...?

We'll sumarize the results so far as follows:

* When a role and resource are in the same account permssion is granted if either the role or resource grants access. This is called union.

* When a role and resource are in different accounts, permission must be granted by both the role and the resource. This is called intersection.

# Now comes a challenge...

Notice that we haven't talked about bucket5? Well, that's because bucket5 is an optional challenge you can try to solve (if you feel you can, that's it).

For this challenge you'll need to know three things:

* The bucket's name is correlated to your username

* The bucket's permissions are divorced

* In order to download an item from a bucket, you do something like the following, where you specify what you want to download (using the cp command) and the name you want to give it after it downloads

```!./aws_run_as.sh --profile lab2 $roleA s3 cp s3://exampleusername-bucket-4/example.txt example.txt```

Use the cells below to write the necessary commands. If you need to, you can add more cells by pressing A or B on your keyboard when the selected cell is blue.